In [30]:
path = 'X:\CODER\ParserService\requirements.txt'
import asyncio
import httpx
from uuid import UUID, uuid4
from pydantic import BaseModel, Field

from sensory_data_client import DataClient, get_settings, DataClientConfig, PostgresConfig, MinioConfig, create_data_client
from sensory_data_client.models.document import DocumentCreate, DocumentMetadata

In [31]:
#config = DataClientConfig(PostgresConfig(host=5422))

In [32]:
client.upload_file('X:\CODER\ParserService\X5Group x SensoryLAB 07.02.2024.pptx.pdf', )

TypeError: DataClient.upload_file() missing 2 required positional arguments: 'content' and 'meta'

In [ ]:
import asyncio
from pathlib import Path  # Используем pathlib для удобной и кросс-платформенной работы с путями
import uuid
from typing import Optional, Coroutine, Any

from pydantic import BaseModel, Field

client =create_data_client(DataClientConfig(postgres = PostgresConfig(user = "postgres",
                                                           password = "postgres",
                                                           host = "localhost",
                                                           port=5422), 
                                            minio = MinioConfig(endpoint="localhost:9008",  bucket = "corporate")))


endpoint='localhost:9008' accesskey='minioadmin' secretkey='minioadmin' bucket='corporate' secure=False


In [43]:

# Шаг 1: Определяем путь к файлу
# Использование pathlib.Path предпочтительнее, чем простые строки.
# Он сам разберется со слэшами в Windows/Linux.
file_path_str = r'X:\CODER\ParserService\tests\Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf'
file_path = Path(file_path_str)

# Проверка, что файл существует, перед тем как его читать
if not file_path.is_file():
    print(f"Ошибка: Файл не найден по пути {file_path}")

print(f"Подготовка к загрузке файла: {file_path}")

# Шаг 2: Читаем файл в байты
# .read_bytes() - простой способ прочитать весь файл в память
try:
    file_content: bytes = file_path.read_bytes()
except Exception as e:
    print(f"Не удалось прочитать файл: {e}")
    

DOC_ID = uuid4()
# Шаг 3: Создаем объект метаданных DocumentCreate
# Извлекаем информацию из пути и добавляем остальную
document_meta = DocumentCreate(
    user_document_id=str(uuid.uuid4()), # Генерируем уникальный ID для этого документа
    name=file_path.name,                # 'X5Group x SensoryLAB 07.02.2024.pptx.pdf'
    owner="user-alpha-007",             # ID пользователя, который загружает файл
    access_group="x5-auditors",         # Опциональная группа доступа
    metadata=DocumentMetadata()
)


Подготовка к загрузке файла: X:\CODER\ParserService\tests\Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf


In [45]:

# Шаг 4: Вызываем асинхронный метод с `await`
try:
    # Ваш метод `upload_file` скорее всего захочет получить чистое имя файла,
    # а не весь путь. file_path.name идеально для этого подходит.
    result_document = await client.upload_file(
        file_name=file_path.name,
        content=file_content,
        meta=document_meta
    )
    
    print("\n✅ Успешно загружен документ!")
    print(f"   ID в базе: {result_document.id}")
    print(f"   Путь в хранилище: {result_document}")

except Exception as e:
    print(f"❌ Произошла ошибка во время загрузки: {e}")



✅ Успешно загружен документ!
   ID в базе: 578b2be3-beb7-4545-8ba8-bea280253ce8
   Путь в хранилище: user_document_id='50424754-290d-4667-94c8-e06c95857fc2' name='Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf' owner='user-alpha-007' access_group='x5-auditors' metadata=DocumentMetadata(processing_status=None, image_object_paths=None, extra=None) id=UUID('578b2be3-beb7-4545-8ba8-bea280253ce8') created=datetime.datetime(2025, 8, 6, 22, 20, 7, 363341, tzinfo=datetime.timezone.utc) edited=datetime.datetime(2025, 8, 6, 22, 20, 7, 363341, tzinfo=datetime.timezone.utc) content_hash='a5fef74e0b09c41553613ed214c243d12c41f769d98edd2b9a31f8db1e112bb5' object_path='pdf/578b2be3beb745458ba8bea280253ce8/Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf' extension=None


In [ ]:
raw_bytes = await client.get_file(UUID('0e215786-b1d0-4989-aad7-801c44af70a0'))

In [47]:
from parser_client import DocParserClient
DOC_ID = '578b2be3-beb7-4545-8ba8-bea280253ce8'
PARSER_URL = "http://localhost:8000" # URL нашего doc-parser сервиса
FILE_NAME = "Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf"

# 2. Запустить парсинг и дождаться результата
parser_client = DocParserClient(base_url=PARSER_URL)
try:
    result = await parser_client.parse_and_wait(doc_id=DOC_ID, file_name=FILE_NAME)
    print("\n--- Final Result ---")
    print(result) # -> {'lines_count': 150, 'images_count': 3}
except (asyncio.TimeoutError) as e:
    print(f"\n--- Error --- \n{e}")

Client: Starting parsing for doc_id=578b2be3-beb7-4545-8ba8-bea280253ce8, file_name='Акт_сдачи_приемки_работ_оказанных_услуг_№1_от_21_07_2025.pdf'...
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for 578b2be3-beb7-4545-8ba8-bea280253ce8... [ 50%] IN_PROGRESS | Stage: PARSING
Client: Polling status for